# Modern Methods for Text Classification

Text Classification is a very frequently seen challenge. This has several applications ranging from sentiment analysis, tagging and filtering news articles, and detecting fraud reviews (on websites such as Amazon) to name a few. 

For simplicity, we will be working with a sentiment analysis dataset. For same reasons, we evaluate accuracy as our only metric here. 

We have a curated set of movie reviews picked up from Imdb. Each review is marked as either positive or negative. Ofcourse, this marks the overall sentiment of each review. 

Beginning this section, we will incorporate more and more Machine Learning instead of relying on linguistics analysis alone. In writing this, I assume that you have some basic familiarity with Python packages like [scikit-learn](http://scikit-learn.org/). 

If you don't, that's fine too. The intent here is too give you a quick reference of how these APIs functions work and save your time in looking up what to learn. 

You can and must learn to use such functions well, even without knowing all the nitty gritty of underlying math. You can trust these functions as black boxes.

### Simple Classifiers

We begin by simply tries a few machine learning classifiers such as Logistic Regression, Naive Bayes, Decision Trees. 
Next, we try Random Forest and Extra Trees Classifier. For all of these implementations, we don't use anything except scikit-learn. 


### Optimizing Simple Classifiers

We can tweak the simple classifiers above to improve their performance. For this, the most common method is to try several slightly different versions of the classifier. We do this by changing the parameters of our classifier. 

We will learn how to automate this "search" process for the best classifier parameters using *GridSearch* and *RandomizedSearch*

### Ensemble Methods

Ensemble several different classifiers means we will be using a group of models. It is a very popular and easy to understand machine learning technique. This is part of almost every winning Kaggle competition. 

Despite initial concerns of why this might be slow, some teams working on commercial software have begun using this in production software as well. This is because it requires very little overhead, is easy to parallelize, and allows for a built-in fallback of using a single model. 

We will look at some of the simplest ensembling techniques based on simple majority, also known as voting ensemble and build using that.

In summary, this **Machine Learning for NLP** section covers simple classifiers, parameter optimization, and ensemble methods

In [1]:
from pathlib import Path
import pandas as pd
import gzip
from urllib.request import urlretrieve
from tqdm import tqdm
import os
import numpy as np
# if you are using the fastAI environment, all of these imports work

In [2]:
class TqdmUpTo(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None: self.total = tsize
        self.update(b * bsize - self.n)

In [3]:
def get_data(url, filename):
    """
    Download data if the filename does not exist already
    Uses Tqdm to show download progress
    """
    if not os.path.exists(filename):

        dirname = os.path.dirname(filename)
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        with TqdmUpTo(unit='B', unit_scale=True, miniters=1, desc=url.split('/')[-1]) as t:
            urlretrieve(url, filename, reporthook=t.update_to)

In [4]:
# Let's download some data:
data_url = 'http://files.fast.ai/data/aclImdb.tgz'
get_data(data_url, 'data/imdb.tgz')

Let's extract the files above and see what the directory contains:

In [5]:
data_path = Path(os.getcwd())/'data'/'imdb'/'aclImdb'
assert data_path.exists()
for pathroute in os.walk(data_path):
    next_path = pathroute[1]
    for stop in next_path:
        print(stop)

test
train
all
neg
pos
all
neg
pos
unsup


This really badly written utility tells us that there are atleast two folders: `train` and `test`. Each of these folders in turn has atleast 3 folders:
```bash
Test
|- all
|- neg
|- pos
```
and

```bash
Train
|- all
|- neg
|- pos
|- unsup
```

The pos and neg folders contain reviews which are positive and negative respectively. The `unsup` folder stands for unsupervised. They are useful for building language models, specially for Deep Learning. We will not use that here. Similarly, the folder `all` is redundant because these reviews are repeated in pos and neg folders. 

# Read Data 

In [6]:
train_path = data_path/'train'
test_path = data_path/'test'

In [7]:
def read_data(dir_path):
    """read data into pandas dataframe"""
    
    def load_dir_reviews(reviews_path):
        files_list = list(reviews_path.iterdir())
        reviews = []
        for filename in files_list:
            f = open(filename, 'r', encoding='utf-8')
            reviews.append(f.read())
        return pd.DataFrame({'text':reviews})
        
    
    pos_path = dir_path/'pos'
    neg_path = dir_path/'neg'
    
    pos_reviews, neg_reviews = load_dir_reviews(pos_path), load_dir_reviews(neg_path)
    
    pos_reviews['label'] = 1
    neg_reviews['label'] = 0
    
    merged = pd.concat([pos_reviews, neg_reviews])
    merged.reset_index(inplace=True)
    
    return merged

In [8]:
train = read_data(train_path)
test = read_data(test_path)

In [9]:
X_train, y_train = train['text'], train['label']
X_test, y_test = test['text'], test['label']

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

## Logistic Regression
The simplest of all, we replicate the exact steps which we saw from Chapter 01. 

Feature Extraction: 
- Bag of Words
- TF-IDF

In [11]:
from sklearn.linear_model import LogisticRegression as LR

In [12]:
lr_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf',LR())])

We saw the Pipeline in our introductory section. Pipeline allows to queue multiple operations in one single Python object.

#### !TIP
We are able to call functions like `fit`, `predict` and `fit_transform` on our `Pipeline` objects because Pipeline automatically calls the corresponding function of the last component in the list 

In [13]:
%%time
lr_clf.fit(X=X_train, y=y_train) # note that .fit function calls are inplace, and the Pipeline is not re-assigned

Wall time: 5.71 s


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

TK mention fit, fit_transform and partial_fit here 
- add code examples for partial fit here

In [14]:
lr_predicted = lr_clf.predict(X_test)

As mentioned earlier, we are calling the `predict` function on our Pipeline. The test reviews go through under the same pre-processing steps e.g. `CountVectorizer()` and `TfidfTransformer()` here as the reviews during training. 

This ease of simplicity makes `Pipeline` one of the most frequently used abstractions in software-grade machine learning. Users might prefer to execute each step independently, or build their own Pipeline equivalents in some research/experimentation use cases. 

In [15]:
lr_acc = sum(lr_predicted == y_test)/len(lr_predicted)
lr_acc

0.88316

**How do we find our model accuracy?**
Let's take a quick look at what is happening in the line above. 

Consider that our predictions are: `[1, 1, 1]` and ground truth: `[1, 0, 1]`. The equality would return a simple list of boolean objects like: `[True, False, True]`. When we `sum` a boolean list in Python, it returns the number of True cases - giving us exact count of how many times did our model make correct predictions. 

Diving this value by the total number of predictions made (or, equally the number of test reviews) gives us our accuracy.

Let's write the above two line logic into a simple, light weight function to calculate accuracy. This would prevent us from repeating the logic.   

In [16]:
def imdb_acc(pipeline_clf):
    predictions = pipeline_clf.predict(X_test)
    assert len(y_test) == len(predictions)
    return sum(predictions == y_test)/len(y_test), predictions

### Remove Stop Words

In [17]:
lr_clf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()), ('clf',LR())])

In [18]:
lr_clf.fit(X=X_train, y=y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [19]:
lr_acc, lr_predictions = imdb_acc(lr_clf)
lr_acc

0.879

### Increase the Ngram Range

In [20]:
lr_clf = Pipeline([('vect', CountVectorizer(stop_words='english', ngram_range=(1,3))), ('tfidf', TfidfTransformer()), ('clf',LR())])

In [21]:
lr_clf.fit(X=X_train, y=y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [22]:
lr_acc, lr_predictions = imdb_acc(lr_clf)
lr_acc

0.86596

# Multinomial Naive Bayes

Note: Why is the above called Naive? There are more powerful and complex methods involving Bayesian approaches. 

In [23]:
from sklearn.naive_bayes import MultinomialNB as MNB
mnb_clf = Pipeline([('vect', CountVectorizer()), ('clf',MNB())])

In [24]:
mnb_clf.fit(X=X_train, y=y_train)
mnb_acc, mnb_predictions = imdb_acc(mnb_clf)
mnb_acc

0.81356

### Add TF-IDF

Now, let's try the above model with TF-IDF as another step after the Bag of Words (Unigrams)

In [25]:
mnb_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf',MNB())])

In [26]:
mnb_clf.fit(X=X_train, y=y_train)
mnb_acc, mnb_predictions = imdb_acc(mnb_clf)
mnb_acc

0.82956

### Remove Stop Words

In [27]:
mnb_clf = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()), ('clf',MNB())])

In [28]:
mnb_clf.fit(X=X_train, y=y_train)
mnb_acc, mnb_predictions = imdb_acc(mnb_clf)
mnb_acc

0.82992

### Add Ngram Range from 1 to 3

In [29]:
mnb_clf = Pipeline([('vect', CountVectorizer(stop_words='english', ngram_range=(1,3))), ('tfidf', TfidfTransformer()), ('clf',MNB())])

In [30]:
mnb_clf.fit(X=X_train, y=y_train)
mnb_acc, mnb_predictions = imdb_acc(mnb_clf)
mnb_acc

0.8572

### Change Fit Prior to False

In [31]:
mnb_clf = Pipeline([('vect', CountVectorizer(stop_words='english', ngram_range=(1,3))), ('tfidf', TfidfTransformer()), ('clf',MNB(fit_prior=False))])

In [32]:
mnb_clf.fit(X=X_train, y=y_train)
mnb_acc, mnb_predictions = imdb_acc(mnb_clf)
mnb_acc

0.8572

In the above example, we made small modifications to try out few combinations in our Pipeline. 

We thought of each combination which might improve our performance. Increasing the `ngram_range` did work, while changing prior from uniform to fitting it (by changing `fit_prior` to False) did not help at all. This approach is tedious, and slightly error-prone because it also relies too much on human intuition of underlying data the machine learning model to be correct.

### Why we don't try Gaussian Naive Bayes?

Gaussian Naive Bayes assumes that the underlying features matrix (our TF-IDF) is densely packed. Owing to the nature of text (where every word is a feature), this is not the case. Our TF-IDF matrix is not densely packed. 

Additionally, our feature matrix is not even close to a Gaussian distribution.  

We don't use Gaussian Naive Bayes for text classification, because it would not meet our requirements and assumptions. 

### Support Vector Machine
Prior work such as that by [T Joachims](https://www.cs.cornell.edu/people/tj/publications/joachims_98a.pdf) with over 9K citations recommend Support Vector Classifiers for text classification. 

It's difficult to estimate whether it will be equally effective for us or not based on such literature due to difference in dataset, pre-processing steps. Let's give it a shot nevertheless:

In [33]:
from sklearn.svm import SVC

In [34]:
svc_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf',SVC())])

In [35]:
# %%time
# svc_clf.fit(X=X_train, y=y_train)
# Wall time: 14min 23s

In [36]:
# %%time
# svc_acc, svc_predictions = imdb_acc(svc_clf)
# print(svc_acc)
# 0.6562
# Wall time: 13min 4s

While SVM works best with linearly separabale data (looks like our text is usually not linearly separable), we still wanted to give it a try for completeness. 

Here, SVM does not have a great performance, and took a really long time to train (~150x) of most other classifiers. We will not look at SVM for this particular dataset again.

## Tree Baseed Models

### Decision Trees

In [37]:
from sklearn.tree import DecisionTreeClassifier as DTC
dtc_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf',DTC())])

In [99]:
dtc_clf.fit(X=X_train, y=y_train)
dtc_acc, dtc_predictions = imdb_acc(dtc_clf)
dtc_acc

0.70356

## Random Forest Classifier 

In [39]:
from sklearn.ensemble import RandomForestClassifier as RFC
rfc_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf',RFC())])

In [98]:
rfc_clf.fit(X=X_train, y=y_train)
rfc_acc, rfc_predictions = imdb_acc(rfc_clf)
rfc_acc

0.73096

## Extra Trees Classifier 

In [41]:
from sklearn.ensemble import ExtraTreesClassifier as XTC
xtc_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf',XTC())])

In [97]:
xtc_clf.fit(X=X_train, y=y_train)
xtc_acc, xtc_predictions = imdb_acc(xtc_clf)
xtc_acc

0.74832

# Automatically Fine Tuning 

Let's focus on our best performing model: Logistic Regression and see if we can push it's performance a little more. 

The best performance for our model was **0.88312** accuracy earlier. 

We are using the phrases parameter-search and hyperparameter search interchangeably here. This is done to stay consistent with the Deep Learning vocabulary.

We want to select the best performing configuration of our pipeline. Each configuration might be diffirent is small ways like removing stop words, including bigrams and trigrams or similar. 

The total number of such configurations can be fairly large running into few thousands. In addition to manually selecting few combinations to try, we can try all of these several thousand combinations *and* evaluate each combination. 

This is too slow for most small-scale experiments such as ours. In large experiments, the possible space can run into millions and several days of computing again making it cost and time prohibitive. 

I strongly urge you to read this blog on [Hyperparameter Tuning](https://www.oreilly.com/ideas/evaluating-machine-learning-models/page/5/hyperparameter-tuning) to become familiar with the vocabulary and ideas in the space beyond what is discussed here. 

### RandomizedSearch

An alternative was proposed by [*Bergstra & Bengio, 2012*](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf). They demonstrated that Random Search across a large hyperparameter space is more effective than manual (as we did for Multinomial Naive Bayes) and often as-effective or more effective than Grid Search. 

**How do we use it here?**
We build on top of the results such as that of Bergstra et al. We break down our parameter search into two steps: 
Step 1: Randomized Search to go through a wide parameter combination space in a limited number of iterations 
Step 2: Use the results above to run a GridSearch in that slightly narrow space. 

We can repeat the above steps till we stop seeing improvements in our results, but we won't do that here. We leave that as an exercise to the reader

In [43]:
from sklearn.model_selection import RandomizedSearchCV

#### How to prepare the param_grid?
TK

In [44]:
param_grid = dict(clf__C=[50, 75, 85, 100], 
                  vect__stop_words=['english', None],
                  vect__ngram_range = [(1, 1), (1, 3)],
                  vect__lowercase = [True, False],
                 )

In [45]:
random_search = RandomizedSearchCV(lr_clf, param_distributions=param_grid, n_iter=5, scoring='accuracy', n_jobs=-1, cv=3)

What does cv do? Adding cv above causes use of StratifiedKFold for evaluation of the scoring metric
What does n_iter do? 
What does scoring do? 

In [46]:
%%time
random_search.fit(X_train, y_train)

Wall time: 3min 7s


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words='english',
        ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
          fit_params=None, iid=True, n_iter=5, n_jobs=-1,
          param_distributions={'clf__C': [50, 75, 85, 100], 'vect__stop_words': ['english', None], 'vect__ngram_range': [(1, 1), (1, 3)], 'vect__lowercase': [True, False]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [47]:
print(f'Calculated cross-validation accuracy: {random_search.best_score_}')

Calculated cross-validation accuracy: 0.87616


In order to compare the performance of this classifier on the ones which we have already seen, we need to train it on complete dataset and test it on the same split as earlier. We do this next:

In [48]:
best_random_clf = random_search.best_estimator_

In [49]:
best_random_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [50]:
imdb_acc(best_random_clf)

(0.90052, array([1, 1, 1, ..., 0, 0, 1], dtype=int64))

We see that the classifier performance improves by more than 1% by simply changing very few parameters. This is amazing. 

Let's see what parameters are here. In order to compare this, you would need to know the default values for all of the parameters. Alternatively, we can simply look at the parameters from the `param_grid` that we wrote and note the selected parameter values. For everything not in the grid, default values are chosen and remain unchanged.

In [51]:
best_random_clf.steps

[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 3), preprocessor=None, stop_words=None,
          strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=None, vocabulary=None)),
 ('tfidf',
  TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
 ('clf',
  LogisticRegression(C=50, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

We notice that in the best classifier: 
    - the chosen C value in clf is 100, 
    - lowercase is set to False
    - removing stop words is bad idea, and
    - adding bigrams and trigrams helps
    
Observations like these are very specific to this dataset and classifier pipeline. In my experience, this can and does vary widely.

We can also not assume that the values are always the best value when we run `RandomizedSearch` for so few iterations. The rule of thumb is to run it for **60 iterations** atleast, and use a much larger `param_grid` as well. 

We used RandomizedSearch to understand the broad layout of parameters we want to try. We add the best values for some of those to our pipeline itself and continue to experiment with values of other parameters. 

We will now run GridSearch for these selected parameters. Here, on a whim, I am choosing to include bigrams and trigrams while running grid search over the `parameter C` of LogisticRegression. 

**!TIP**

I have not mentioned what the parameter `C` stands for or how it influences the classifier. This is definitely important to understand while doing manual parameter search. I could notice that changing `C` helps simply by trying out different values. 

But, our intention here is to automate as much as possible. I instead try varying values in `C` to try during our `RandomizedSearch`. We are trading off human learning time (maybe a few hours) with compute time (maybe a few extra minutes). This mindset saves us time and effort both.

In [52]:
lr_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,3))), ('tfidf', TfidfTransformer()), ('clf',LR())])

In [53]:
from sklearn.model_selection import GridSearchCV

In [54]:
param_grid = dict(clf__C=[85, 100, 125, 150])
grid_search = GridSearchCV(lr_clf, param_grid=param_grid, scoring='accuracy', n_jobs=-1, cv=3)

In [55]:
%%time
grid_search.fit(X_train, y_train)

Wall time: 5min 32s


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__C': [85, 100, 125, 150]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='accuracy',
       verbose=0)

In [56]:
grid_search.best_estimator_.steps

[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 3), preprocessor=None, stop_words=None,
          strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=None, vocabulary=None)),
 ('tfidf',
  TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
 ('clf',
  LogisticRegression(C=150, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
            verbose=0, warm_start=False))]

In [57]:
print(f'Calculated cross-validation accuracy: {grid_search.best_score_} while random_search was {random_search.best_score_}')

Calculated cross-validation accuracy: 0.87684 while random_search was 0.87616


In [58]:
%%time
best_grid_clf = grid_search.best_estimator_
best_grid_clf.fit(X_train, y_train)

Wall time: 1min 9s


In [59]:
imdb_acc(best_grid_clf)

(0.90208, array([1, 1, 1, ..., 0, 0, 1], dtype=int64))

That's almost ~3% performance gain over the unoptimized model. This was despite the fact that we tried very few parameters to optimize itself. 

It is worth mentioning that we can and must repeat these steps (RandomizedSearch and GridSearch) to push the model accuracy even further. 

# Ensemble Models 
**Ensembling models** is a very powerful technique to improve your model performance across a variety of Machine Learning tasks. 

In the section below, I borrow heavily from the [Kaggle Ensembling Guide](https://mlwave.com/kaggle-ensembling-guide/) written by [MLWave](https://mlwave.com/).

I explain why ensembling helps reduce error, or improve accuracy. I demonstrate all the popular techniques on our chosen task and dataset. While each of these techniques might not result in a performance gain for us specifically on our dataset, they are a powerful tool in your mental toolkit.  

To ensure that you understand these techniques, I strongly urge you to try them on a few datasets. 

## Voting Ensemble

### Simple Majority (aka Hard Voting)
The simplest ensembling technique is perhaps to take a simple majority. This works on the intuition that a single model might make a error on a particular prediction, but several different models are unlikely to make identical errors. 

Let's look at an example. 

Ground truth: 1**1**011001

Let's assume there are 3 models with only one error for this example

Model A Prediction: 1**0**011001

Model B Prediction: 1**1**011001

Model C Prediction: 1**1**011001

The majority votes gives us the correct answer in this example - 

Majority vote: 1**1**10110011

---

To try this on our dataset, we import the VotingClassifier from scikit-learn. VotingClassifier does not use the pre-trained models as inputs. It will call fit on the models or classifier pipelines, and use the predictions of all models to make the final prediction. 

To counter the hype in favour of ensembles elsewhere, we demonstrate that hard voting can actually hurt your accuracy performance. If someone claims that ensembling _always_ helps, you can quickly point them here to have a more constructive discussion. 

In [60]:
from sklearn.ensemble import VotingClassifier

In [67]:
%%time
voting_clf = VotingClassifier(estimators=[('xtc', xtc_clf), ('rfc', rfc_clf)], voting='hard', n_jobs=-1)
voting_clf.fit(X_train, y_train)

Wall time: 18.9 s


In [69]:
hard_voting_acc, _ = imdb_acc(voting_clf)
hard_voting_acc

C:\Users\nirantk\Anaconda3\envs\fastai\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.71684

Using only two classifiers for demonstration, we use the eXtra Trees and Random Forest Classifiers. Individually, each of these classifiers have their performance capped at ~74% accuracy. 

In this particular example, the performance of the voting classifier is worse than both of them alone. 

#### Soft Voting

Soft voting predicts the class label based on class probabilities. The sums of the predicted probabilities for each classifier is calculated for each class (important, in case of multi-class). The assigned class is then the class with maximum probability sum or argmax(p_sum). 

This is recommended for an ensemble of well-calibrated classifiers. 

    Well calibrated classifiers are probabilistic classifiers for which the output of the predict_proba method can be directly interpreted as a confidence level. - from the [Callibration Docs on sklearn](http://scikit-learn.org/stable/modules/calibration.html)

In [72]:
%%time
voting_clf = VotingClassifier(estimators=[('lr', lr_clf), ('mnb', mnb_clf)], voting='soft', n_jobs=-1)
voting_clf.fit(X_train, y_train)

Wall time: 1min 27s


In [107]:
import warnings
warnings.filterwarnings('ignore')

In [108]:
soft_voting_acc, _ = imdb_acc(voting_clf)
soft_voting_acc

0.88216

In [81]:
gain_acc = soft_voting_acc - lr_acc
if gain_acc > 0:
    print(f'We see that the soft voting gives us an absolute accuracy gain of {gain_acc*100:.2f}% ')

We see that the soft voting gives us an absolute accuracy gain of 1.62% 


### Weighted Classifiers

The only way for the inferior models to overrule the best model (expert) is for them to collectively (and confidently) agree on an alternative. 

- How can we avoid this scenario? We then use a weighted majority vote
- Why weighing? Usually we want to give a better model more weight in a vote

In [93]:
%%time
weighted_voting_clf = VotingClassifier(estimators=[('lr', lr_clf), ('lr2', lr_clf),('rf', xtc_clf), ('mnb2', mnb_clf),('mnb', mnb_clf)], voting='soft', n_jobs=-1)
weighted_voting_clf.fit(X_train, y_train)

Wall time: 1min 41s


Repeat the experiment with 'hard' voting instead of 'soft' voting. This will tell you how does the voting strategy influence the accuracy of our ensembled classifier. 

In [94]:
weighted_voting_acc, _ = imdb_acc(weighted_voting_clf)
weighted_voting_acc
# 0.87996 for soft

C:\Users\nirantk\Anaconda3\envs\fastai\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.88092

In [95]:
gain_acc = weighted_voting_acc - lr_acc
if gain_acc > 0:
    print(f'We see that the weighted voting gives us an absolute accuracy gain of {gain_acc*100:.2f}%')

We see that the weighted voting gives us an absolute accuracy gain of 1.50%


##### What have we learnt so far?  
- Simple majority based voting classifier can perform worse than individual models 
- Soft Voting works better than Hard voting
- Weighing classifiers by simply repeating the classifiers can help

So far, we have been selecting classifiers seemingly at random. This is less than ideal, specially when we are builing for commerical utility where very 0.001% gain matters. 

#### Removing Correlated Classifiers 

To see this, let us take 3 simple models again. The ground truth is all 1’s:

`
1111111100 = 80% accuracy
1111111100 = 80% accuracy
1011111100 = 70% accuracy
`

These models are highly correlated in their predictions. When we take a majority vote we see no improvement:

`
1111111100 = 80% accuracy
`

Now we compare to 3 less-performing, but highly uncorrelated models:

`
1111111100 = 80% accuracy
0111011101 = 70% accuracy
1000101111 = 60% accuracy
`

When we ensemble this with a majority vote we get:

`
1111111101 = 90% accuracy
`

We get an improvement which is much higher than any of our individual models. Low correlation between model predictions can lead to better performance. 

In practice, this is tricky to get right but worth investigating nevertheless. 

We leave this as an exercise for you to try. 

As a quick hint, you will need to find the correlations among predictions of different models and select pairs which are less correlated to each other (ideally less than 0.5) and yet having a high enough performance as individual models. 

In [96]:
np.corrcoef(mnb_predictions, lr_predictions)[0][1] # this is too high a correlation

0.8442355164021454

In [109]:
%%time
corr_voting_clf = VotingClassifier(estimators=[('lr', lr_clf), ('mnb', mnb_clf)], voting='soft', n_jobs=-1)
corr_voting_clf.fit(X_train, y_train)
corr_acc, _ = imdb_acc(corr_voting_clf)
print(corr_acc)

0.88216
Wall time: 1min 49s


In [102]:
np.corrcoef(dtc_predictions,xtc_predictions )[0][1] # this is looks like a low correlation

0.3272698219282598

In [110]:
%%time
low_corr_voting_clf = VotingClassifier(estimators=[('dtc', dtc_clf), ('xtc', xtc_clf)], voting='soft', n_jobs=-1)
low_corr_voting_clf.fit(X_train, y_train)
low_corr_acc, _ = imdb_acc(low_corr_voting_clf)
print(low_corr_acc)

0.70564
Wall time: 58.1 s


In Summary
---

We saw several new ideas from machine learning. The intention is here to demonstrate some of the most common classifiers. We see how to use them with one thematic idea: Translate text to a numerical representation and then feed to these classifier.

This covers a very miniscule fraction of the possibilities which you can try, ranging from better feature extraction than Tfidf to tuning classifiers with GridSearch+RandomizedSearch as well as ensembling several classifiers.

What Next?
---
This chapter was mostly focussed on pre-deep learning methods for both feature extraction and classification. 

Deep Learning methods allow us to use a single model, where the feature extraction and classification are both "learned" fromo the underlying data distribution. While a lot has been written about Deep Learning in Computer Vision, we do a very shallow, first introduction for Deep Learning in Natural Language Processing. 